In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as F

text = open('../data/shakespeare.txt', encoding="utf-8").read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

# hyperparameters
block_size = 128
n_embd = 128
n_head = 4
n_layer = 4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 16
eval_iters = 100


# creating mapping between tokens and chars
itoc = {i: c for i, c in enumerate(chars)}
ctoi = {c: i for i, c in enumerate(chars)}
encode = lambda s: [ctoi[c] for c in s]
decode = lambda l: ''.join([itoc[i] for i in l])

data = torch.tensor(encode(text))
n = int(.9*len(data))
train_data = data[:n]
val_data = data[n:]


def get_batch(split):
    
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    X = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    X, y = X.to(device), y.to(device)
    return X, y

@torch.no_grad()
def estimate_loss():
    model.eval()
    out = {}
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, y = get_batch(split)
            logits, loss = model(X, y)
            losses[k] = loss
        out[split] = losses.mean()
    model.train()
    return out


class Head(nn.Module):
    
    def __init__(self, head_size):
        super().__init__()
        
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        
    def forward(self, x):
        
        B,T,C = x.shape
        
        k = self.key(x)
        q = self.query(x)
        
        # compute attention scores (affinities)
        
        wei = q @ k.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))
        wei = F.softmax(wei, dim=-1)
        
        v = self.value(x)
        out = wei @ v
        return out
    
class MultiHeadAttention(nn.Module):
    
    def __init__(self, head_size):
        super().__init__()
        
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_head)])
        self.proj = nn.Linear(n_embd, n_embd)
        
    def forward(self, x):
        
        x = torch.cat([h(x) for h in self.heads], dim=-1)
        x = self.proj(x)
        return x
    
class CausalSelfAttention(nn.Module):
    
    def __init__(self, head_size):
        super().__init__()
        
        self.c_attn = nn.Linear(n_embd, 3*n_embd, bias=False)
        self.proj = nn.Linear(n_embd, n_embd)
        self.register_buffer('tril', torch.tril(torch.ones(1, 1, block_size, block_size)))
        
    def forward(self, x):
        
        B,T,C = x.shape
        
        k,q,v = self.c_attn(x).split(n_embd, dim=-1)
        
        key = k.view(B,T,n_head, n_embd // n_head).transpose(1, 2) # (B, n_head, T, C)
        query = q.view(B,T,n_head, n_embd // n_head).transpose(1, 2) # (B, n_head, T, C)
        value = v.view(B,T,n_head, n_embd // n_head).transpose(1, 2) # (B, n_head, T, C)
        
        wei = query @ key.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:, :, :T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        
        out = wei @ value
        
        out = out.transpose(1,2).contiguous().view(B,T,C)
        out = self.proj(out)
        return out
        
    
class FeedForward(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd, n_embd)
        )
        
    def forward(self, x):
        
        x = self.net(x)
        return x
    
class Block(nn.Module):
    
    def __init__(self):
        
        super().__init__()
        
        head_size = n_embd // n_head
        
        self.sa = CausalSelfAttention(head_size)
        self.ffwd = FeedForward()
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        
    def forward(self, x):
        
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        
        return x
        
        
class Transformer(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        
        self.blocks = nn.Sequential(*[Block() for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    def forward(self, idx, targets=None):
        
        B, T = idx.shape
        
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T).to(device))
        x = tok_emb + pos_emb
        
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        if targets == None:
            loss = None
        else:
            
            B,T,C = logits.shape
            
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx    
    
model = Transformer().to(device)
print(sum(p.numel() for p in model.parameters()) / 1e6, ' M parameters')
optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-3)

0.824897  M parameters


In [34]:
Epochs = 10

for epoch in range(Epochs):
    
    if epoch % (Epochs // 10) == 0 or epoch == 0:
        out = estimate_loss()
        print(f"epoch = {epoch}, train_loss = {out['train']}, val_loss = {out['val']}")
        
    X, y = get_batch('train')
    
    logits, loss = model(X, y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    


epoch = 0, train_loss = 4.366088390350342, val_loss = 4.373683929443359
epoch = 1, train_loss = 3.889199733734131, val_loss = 3.907092809677124
epoch = 2, train_loss = 3.6257259845733643, val_loss = 3.6475272178649902
epoch = 3, train_loss = 3.4812405109405518, val_loss = 3.504429578781128
epoch = 4, train_loss = 3.3897740840911865, val_loss = 3.4142491817474365
epoch = 5, train_loss = 3.3359949588775635, val_loss = 3.3618247509002686
epoch = 6, train_loss = 3.3038687705993652, val_loss = 3.3206801414489746
epoch = 7, train_loss = 3.2586121559143066, val_loss = 3.3019332885742188
epoch = 8, train_loss = 3.2344141006469727, val_loss = 3.281691789627075
epoch = 9, train_loss = 3.224379301071167, val_loss = 3.2596969604492188


In [27]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))


CABEOS:
ENrrighe th.

HARENCERRY SSTHUT:
Pot offilen pevie uso tethe wheve blste thaw my hars
RTAnd cat uck Shifior binth a be bris? y a horghis ank.
Soursen ins lot tens youris our,
Ifie shon thell eat inear soon kistne.

As Anksoy enin; ghas he dhe me tans fak mamen-n,
The akiringh wipsts, thoo thean'd thaly hot Wouhth;
To bof lin'stor, shthis ant hee pe ige dsct fad sthis sideghan
ora! gre enstist gepeal, as! iffe isstng-d some,
Weaghit oft, hape goornhy, hikeer, whan? allld: se is?

Hecre gr
